# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 수정사항
- 2023.03.21
    - item 2 : EG
        - 1. 개별 차량에 대한 지역코드를 부여하여, 지역마다 얼마나 차량들이 존재하는지 구분이 필요한 것 같음

    이 외에도 내일 오전에 동양에서 가시화와 관련된 회의에서 추가적으로 필요하다고 판단되는 사항에 대해 추가적으로 분석이 필요할 것 같음

- 2023.03.22
    - item2 : EG 
        - result파일에 'EG' 컬럼 추가

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1/raw'
ods_fold = 'D:/data/big2/ODS'
sample_fold = 'D:/data/big2/BD1/sample'
analysis_fold = 'D:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [4]:
# about 12s
name = '4등급 차량 리스트 양식 result 최종(2023.03.22)'
file_name = f'{name}.csv'
result = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153813 entries, 0 to 1153812
Data columns (total 31 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1153813 non-null  object 
 1   차량번호      1153813 non-null  object 
 2   제원관리번호    1153813 non-null  object 
 3   차종_원본     1153813 non-null  object 
 4   용도        1153813 non-null  object 
 5   최초등록일     1153813 non-null  int64  
 6   연식        1153813 non-null  int64  
 7   제작일자      1153809 non-null  float64
 8   검사유효일     1153810 non-null  float64
 9   배출가스인증번호  1146663 non-null  object 
 10  등급        1153813 non-null  int64  
 11  차명        1153813 non-null  object 
 12  차종분류      1153813 non-null  object 
 13  차종유형      1153813 non-null  object 
 14  자동차형식     1153812 non-null  object 
 15  제작사명      1153795 non-null  object 
 16  연료        1153813 non-null  object 
 17  엔진형식      1153813 non-null  object 
 18  총중량       1153813 non-null  int64  
 19  적재중량      1153589 non

## 본거지법정동코드

In [5]:
code = pd.read_excel('D:/data/big2/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


# 전처리

## 4등급 차량만

In [6]:
result['등급_수정'].value_counts(dropna=False)

4    1148315
3       5372
5        126
Name: 등급_수정, dtype: int64

In [7]:
result1 = result.loc[result['등급_수정'] == 4]
result1.shape

(1148315, 31)

In [8]:
g3_g5 = result.loc[result['등급_수정'] != 4]
g3_g5.shape

(5498, 31)

## 중복 차대번호 제거
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [9]:
len(result1['차대번호'].unique())

1148315

## 기타연료 처리
- '제원관리번호'로 매칭
- '자동차형식' 으로 매칭
- '엔진형식' 으로 매칭

In [10]:
result1['연료'].value_counts(dropna=False)

경유    1148315
Name: 연료, dtype: int64

## 본거지법정동 코드 병합
- 본거지법정동코드, 법정동코드 기준

In [11]:
result1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

In [12]:
code.columns

Index(['법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'], dtype='object')

In [13]:
result1[['법정동코드', '시도명', '시군구명', '말소여부']].head()

,법정동코드,시도명,시군구명,말소여부
0,3017011600,대전광역시,서구,N
1,2635010700,부산광역시,해운대구,N
2,4148012200,경기도,파주시,N
3,2623010900,부산광역시,부산진구,N
4,5013010300,제주특별자치도,서귀포시,N


In [14]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y


In [15]:
result1.shape

(1148315, 31)

In [16]:
df = result1.merge(code, on='법정동코드', how='left')
df.shape

(1148315, 39)

# 분석

In [17]:
grade_list = []
for f, y, cy, e in tqdm(result1[['연료', '제작일자', '연식', 'DPF유무_수정']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1148315/1148315 [00:02<00:00, 520616.01it/s]


1148315

In [18]:
grade_list[:5]

['D', 'D', 'D', 'D', 'D']

In [19]:
num = 85420
grade_list[num:num+10]

['C', 'D', 'C', 'D', 'C', 'C', 'D', 'D', 'C', 'C']

In [20]:
grade_list[-5:]

['X', 'X', 'X', 'X', 'X']

In [21]:
df['Grade'] = grade_list

In [22]:
num_by_fuel = df.groupby(['연료'])['차대번호'].count().reset_index().sort_values('차대번호', ascending=False)
num_by_fuel = num_by_fuel.rename(columns={'차대번호':'대수'})
num_by_fuel

,연료,대수
0,경유,1148315


In [23]:
num_by_grade = df.groupby(['연료', 'Grade'])['차대번호'].count()
num_by_grade = num_by_grade.rename('대수')
num_by_grade

연료  Grade
경유  A        164313
    B         85721
    C        840425
    D         39038
    X         18818
Name: 대수, dtype: int64

In [24]:
num_by_grade_by_local = df.groupby(['대기관리권역', '시도명', '시군구명', 'Grade'])['차대번호'].count()
num_by_grade_by_local = num_by_grade_by_local.rename('대수')
num_by_grade_by_local

대기관리권역  시도명   시군구명  Grade
그 외     강원도   강릉시   A         928
                    B         459
                    C        4221
                    D         213
                    X          90
                             ... 
중부권     충청북도  충주시   A         880
                    B         416
                    C        4999
                    D         218
                    X         103
Name: 대수, Length: 1250, dtype: int64

## [출력] df + g3_g5

In [64]:
total = pd.concat([df, g3_g5], ignore_index=True)
total.shape

(1153813, 40)

In [65]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '시도', '시구군', '읍면동', '리', '지역3',
       '폐지여부', '대기관리권역', '대기관리권역YN', 'Grade'],
      dtype='object')

In [66]:
total = total[['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       'Grade']]
total = total.rename(columns={'차종_원본':'차종', 'Grade':'EG'})
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       'EG'],
      dtype='object')

In [76]:
today_date = datetime.today().strftime("%Y.%m.%d")
today_date

'2023.03.23'

In [67]:
# # about 14s
# total.iloc[:500000].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.csv'), index=False, encoding='cp949')
# total.iloc[500000:].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.csv'), index=False, encoding='cp949')

In [68]:
# # about 14s
# total.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date}).csv'), index=False, encoding='cp949')

## [출력] EG 통계

In [69]:
analysis_fold

'D:/data/big2/BD1/an'

In [70]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.03.23_AM.09.29.10'

In [71]:
grade = 4
with pd.ExcelWriter(os.path.join(analysis_fold, f'BD1_G{grade}_통계_EG_{today_date}.xlsx')) as writer:
    num_by_fuel.to_excel(writer, sheet_name='연료별', index=False)
    num_by_grade.to_excel(writer, sheet_name='등급별')
    num_by_grade_by_local.to_excel(writer, sheet_name='지역별')

## [출력] 지역별 샘플

In [72]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '시도', '시구군', '읍면동', '리', '지역3',
       '폐지여부', '대기관리권역', '대기관리권역YN', 'Grade'],
      dtype='object')

In [73]:
df[['법정동코드', '시도명', '시군구명', 'Grade', '차량번호', 'DPF유무_수정', '연식']].head(10)

,법정동코드,시도명,시군구명,Grade,차량번호,DPF유무_수정,연식
0,3017011600,대전광역시,서구,D,21로7811,무,2006
1,2635010700,부산광역시,해운대구,D,23거2347,무,2006
2,4148012200,경기도,파주시,D,04어9316,무,2006
3,2623010900,부산광역시,부산진구,D,14우2622,무,2006
4,5013010300,제주특별자치도,서귀포시,D,07머3269,무,2006
5,4150031021,경기도,이천시,D,58거7827,무,2006
6,4817012100,경상남도,진주시,D,64보3244,무,2006
7,4167025035,경기도,여주시,D,69가2031,무,2006
8,4213010300,강원도,원주시,D,06주6090,무,2006
9,4684035021,전라남도,무안군,D,32어5631,무,2006


In [74]:
df[['법정동코드', '시도명', '시군구명', 'Grade', '차량번호', 'DPF유무_수정', '연식']].head(10).to_excel(os.path.join(analysis_fold, f'샘플_법정동코드_지역_{today_date}.xlsx'), index=False)

In [75]:
df.groupby(['시도명', '시군구명', 'Grade', 'DPF유무_수정'], dropna=False)['차대번호'].count()

시도명   시군구명  Grade  DPF유무_수정
강원도   강릉시   A      유            928
            B      유            459
            C      무           4221
            D      무            213
            X      확인불가          90
                               ... 
충청북도  충주시   A      유            880
            B      유            416
            C      무           4999
            D      무            218
            X      확인불가         103
Name: 차대번호, Length: 1250, dtype: int64

# 코드 마지막